In [0]:
import pandas as pd
import numpy as np
from keras.models import Sequential,Model
from keras.models import Model as KerasModel
from keras.layers import Input, Dense, Activation, Reshape
from keras.layers import Concatenate, Dropout
from keras.layers.embeddings import Embedding
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import OneHotEncoder

emotionsList= ['like','antipathy', 'hostility','love','warmth','loathe','abhor','intimacy','dislike','venom','affection',
               'tenderness','animosity','attachment','infatuation','fondness','hate'] 

#One hot encode the above list
ohe = OneHotEncoder()
X = np.array(emotionsList, dtype=object).reshape(-1, 1)
transformed_X =ohe.fit_transform(X).toarray()
transformed_X #onehot sparse array of shape (17,17) is the result
#As you can see Transformation of categorical variable(s) in to onehot results in very high dimensions.

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1

In [0]:
#To get repeatable results OR To get stable results use seed as below. In this case we get same set of weights/embedings in every run
# If we omit below 4 lines at each run, different weights/embedings are produced.
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

#Define Embedding Size               
embedding_size = int(min(np.ceil((len(emotionsList))/2), 50 ))

emotion_index = {emotion: idx for idx, emotion in enumerate(emotionsList)}
index_emotion = {idx: emotion for emotion, idx in emotion_index.items()}

#Converting the categorical input to an enity embedding using Neural network embedding layer to reduce the dimenstions from 17 to 9
input_model = Input(shape=(1,))
output_model = Embedding(len(emotionsList), embedding_size, name='emotions_embedding')(input_model) 
output_model = Reshape(target_shape=(embedding_size,))(output_model)

model = Model(inputs = input_model, outputs = output_model)
model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Here Weights are the embeddings,so retreving the weights generated 
emotions_layer = model.get_layer('emotions_embedding')
emotions_weights = emotions_layer.get_weights()[0]
print(emotions_weights) # embedding vector of (17,9)

#As you can see below each emotion from emotionsList is transformed in to an embedding weight/vector of size 9 as opposed to 17 with onehot

[[ 2.74238326e-02 -1.01963505e-02  3.13716270e-02 -1.64878368e-03
  -1.10065229e-02  1.29030831e-02 -3.10807824e-02  1.21418461e-02
  -4.18641567e-02]
 [-9.48626921e-03  4.05936278e-02  4.72326763e-02 -4.25748602e-02
   1.62261724e-03 -3.12898532e-02 -2.86918283e-02 -2.45502237e-02
   1.62344053e-03]
 [-3.08156013e-04 -2.55837794e-02 -3.85068282e-02 -3.10104378e-02
   2.75073536e-02  2.66973712e-02 -6.30190223e-03  2.11631171e-02
   2.75010988e-03]
 [-1.50858387e-02  3.67283709e-02  3.98593210e-02  2.54743434e-02
   2.34374292e-02 -3.30721140e-02 -1.88026782e-02  5.64099476e-03
  -1.90675259e-02]
 [ 8.03954899e-05 -4.13990393e-02 -2.62950789e-02 -9.32812691e-05
   6.46386296e-03  7.42943212e-03 -2.41417885e-02  2.87023522e-02
  -2.55922228e-03]
 [ 2.11567171e-02 -2.89756786e-02  2.26801969e-02  4.37210873e-03
  -5.30521944e-03 -2.24559195e-02  7.60009140e-03  9.04453918e-03
   2.80689634e-02]
 [-4.53131199e-02 -7.19735771e-03  1.36564709e-02  4.30800356e-02
  -3.56816277e-02 -3.4936249